# Imports and Package Installations

In [1]:
import getpass
import os

from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv('LANGSMITH_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["PROJECT_ID"] = os.getenv("PROJECT_ID")
os.environ["REGION"] = os.getenv("REGION")

In [3]:
import vertexai

PROJECT_ID = os.environ["PROJECT_ID"]
REGION = os.environ["REGION"]

vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
from google.colab import auth

auth.authenticate_user()

# Embedding Models

In [4]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding_gemini = VertexAIEmbeddings(model_name="gemini-embedding-001")

d:\Downloads\code\code\Extra-Projects\rag-pipeline-eqanun\.venv\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


# LLM Model

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Creating Vector Store

In [7]:
vector_store = InMemoryVectorStore(embedding_gemini)

# Azerbaijani Retrieval (with gemini embedding)

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
from langchain_community.document_loaders import PyPDFLoader

# file_path = "/content/drive/MyDrive/Labour-Code-of-the-Republic-of-Azerbaijan.pdf"
file_path = "/content/drive/MyDrive/cinayet-mecellesi-pdf.pdf"

loader = PyPDFLoader(file_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1536, chunk_overlap=305)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)


prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

KeyboardInterrupt: 

In [ ]:
question = "Cinayət Məcəlləsinin vəzifələri nələrdir?"

response = graph.invoke({"question": question})
print(response["context"])
print(response["answer"])

[Document(id='4fed9875-e88e-4f5c-be41-1c5c3872e469', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-10-15T11:02:03+04:00', 'author': 'Novruz Ferhadov', 'moddate': '2019-10-15T11:02:03+04:00', 'source': '/content/drive/MyDrive/cinayet-mecellesi-pdf.pdf', 'total_pages': 300, 'page': 3, 'page_label': '4'}, page_content='AZƏRBAYCAN RESPUBLİKASININ CİNAYƏT QANUNUNUN VƏZİFƏLƏRİ VƏ PRİNSİPLƏRİ \n  \nM a d d ə  1 . Azərbaycan Respublikasının cinayət qanunu \n  \n1.1. Azərbaycan Respublikasının cinayət qanunu bu Məcəllədən ibarətdir. \n1.2. Bu Məcəllə Azərbaycan Respublikasının Konstitusiyasına, beynəlxalq hüququn hamılıqla \nqəbul edilmiş normalarına və prinsiplərinə əsaslanır. \n1.3. Cinayət məsuliyyətini müəyyən edən və cinayət törətmiş şəxsin cəzalandırılmasını nəzərdə \ntutan qanunlar yalnız bu Məcəlləyə daxil olunduqdan sonra tətbiq edilə bilər. \n  \nM a d d ə  2 . Azərbaycan Respublikasının Cinayət Məcəlləsinin vəzifələri \n  \n2.1

In [ ]:
import textwrap

question = "On dörd yaşı tamam olmuş şəxs hansı cinayətlərə görə məsuliyyətə cəlb edilə bilər?"

response = graph.invoke({"question": question})
wrapped_answer = textwrap.fill(response["answer"], width=100)
print(wrapped_answer)
print("Sources:", response["context"])

On dörd yaşı tamam olmuş şəxs yalnız qəsdən adam öldürməyə, qəsdən sağlamlığa az ağır və ya ağır
zərər vurmağa, adam oğurlamağa, zorlamağa, seksual xarakterli zorakılıq hərəkətlərinə, oğurluğa,
soyğunçuluğa, quldurluğa, hədə-qorxu ilə tələb etməyə, talama məqsədi olmadan qanunsuz olaraq
avtomobil və ya başqa nəqliyyat vasitəsi ələ keçirməyə görə cinayət məsuliyyətinə cəlb edilə bilər.
Ağırlaşdırıcı hallarda əmlakı qəsdən məhv etməyə və ya zədələməyə, terrorçuluğa, adamları girov
götürməyə ağırlaşdırıcı hallarda xuliqanlığa, odlu silahı, döyüş sursatını, partlayıcı maddələri və
qurğuları talamağa və ya hədə -qorxu ilə tələb etməyə, narkotik vasitələri və ya psixotrop maddələri
talamağa və ya hədə -qorxu ilə tələb etməyə, nəqliyyat vasitələrini və ya yolları yararsız vəziyyətə
salmağa görə də cinayət məsuliyyətinə cəlb edilə bilər. Yetkinlik yaşına çatmayanlar cinayət
törətdikdə, onlara cəza təyin edilə bilər və ya tərbiyəvi xarakterli məcburi tədbirlər tətbiq edilə
bilər.
Sources: [Docu

In [ ]:
import textwrap

question = "Səmimi peşmanlıqla bağlı cinayət məsuliyyətindən azad etmə necə baş verir?"

response = graph.invoke({"question": question})
wrapped_answer = textwrap.fill(response["answer"], width=100)
print(wrapped_answer)
print("Sources:", response["context"])

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Böyük ictimai təhlükə törətməyən cinayət törətmiş şəxs könüllü gəlib təqsirini boynuna aldıqda,
cinayətin üstünün açılmasına fəal kömək etdikdə, cinayət nəticəsində dəymiş ziyanı ödədikdə və ya
vurulmuş zərəri başqa yolla aradan qaldırdıqda cinayət məsuliyyətindən azad edilə bilər. Başqa növ
cinayət törətmiş şəxs, bu Məcəllənin 72.1-ci maddəsində nəzərdə tutulmuş şərtlər mövcud olduqda,
yalnız bu Məcəllənin Xüsusi hissəsinin müvafiq maddələrində bilavasitə müəyyən edilmiş hallarda
cinayət məsuliyyətindən azad edilir. Şəxs bu Məcəllənin 72 – 73-2 və 74-1-ci maddələrində nəzərdə
tutulmuş qaydada cinayət məsuliyyətindən yalnız bir dəfə azad edilir.
Sources: [Document(id='29617e7a-6fb2-4adb-8771-0d8b35195aab', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-10-15T11:02:03+04:00', 'author': 'Novruz Ferhadov', 'moddate': '2019-10-15T11:02:03+04:00', 'source': '/content/drive/MyDrive/cinayet-mecellesi-pdf.pdf', 'total_pages': 300, 'page':

In [ ]:
question = "Böyük ictimai təhlükə törətməyən bir cinayət törətmiş şəxs zərərçəkmişlə barışdığı halda cinayət məsuliyyətindən azad edilə bilərmi və hansı şərtlərlə?"
response = graph.invoke({"question": question})
wrapped_answer = textwrap.fill(response["answer"], width=100)
print(wrapped_answer)
print("Sources:", response["context"])

Böyük ictimai təhlükə törətməyən bir cinayət törətmiş şəxs, zərərçəkmiş şəxslə barışdıqda və ona
dəymiş ziyanı ödədikdə və ya vurulmuş zərəri aradan qaldırdıqda cinayət məsuliyyətindən azad edilə
bilər. Lakin, bəzi cinayətlər üçün ziyanın tamamilə ödənilməsi tələb olunur. Digər cinayətlər üçün
isə, ziyanın ödənilməsi ilə yanaşı, dövlət büdcəsinə əlavə ödənişlər də tələb oluna bilər.
Sources: [Document(id='f15d2e31-ff09-473f-acb4-1fa0db9815e9', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-10-15T11:02:03+04:00', 'author': 'Novruz Ferhadov', 'moddate': '2019-10-15T11:02:03+04:00', 'source': '/content/drive/MyDrive/cinayet-mecellesi-pdf.pdf', 'total_pages': 300, 'page': 29, 'page_label': '30'}, page_content='bu qaydada təyin edilir. [83] \n  \nDÖRDÜNCÜ BÖLMƏ \nCİNAYƏT MƏSULİYYƏTİNDƏN VƏ CƏZADAN AZAD ETMƏ \n  \n11-ci fəsil \nCİNAYƏT MƏSULİYYƏTİNDƏN AZAD ETMƏ \n  \nM a d d ə  7 2 . Səmimi peşmanlıqla bağlı cinayət məsuliyyətindən azad

In [ ]:
question = "Zəruri müdafiə həddini aşmaqla adam öldürmə (Maddə 123.1) və ehtiyatsızlıqdan adam öldürmə (Maddə 124.1) cinayətləri üçün nəzərdə tutulan cəzalar arasında nə fərq var?"

response = graph.invoke({"question": question})
wrapped_answer = textwrap.fill(response["answer"], width=100)
print(wrapped_answer)
print("Sources:", response["context"])

Zəruri müdafiə həddini aşmaqla adam öldürmə (Maddə 123.1) iki ilədək islah işləri, azadlığın
məhdudlaşdırılması və ya azadlıqdan məhrum etmə ilə cəzalandırılır. Ehtiyatsızlıqdan adam öldürmə
(Maddə 124.1) isə iki ilədək islah işləri, üç ilədək azadlığın məhdudlaşdırılması və ya üç ilədək
azadlıqdan məhrum etmə ilə cəzalandırılır. Beləliklə, fərq ondadır ki, ehtiyatsızlıqdan adam öldürmə
üçün azadlığın məhdudlaşdırılması və ya azadlıqdan məhrum etmə cəzası daha uzun müddətə ola bilər.
Sources: [Document(id='ba721837-022b-4778-af24-6d50155c53eb', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-10-15T11:02:03+04:00', 'author': 'Novruz Ferhadov', 'moddate': '2019-10-15T11:02:03+04:00', 'source': '/content/drive/MyDrive/cinayet-mecellesi-pdf.pdf', 'total_pages': 300, 'page': 54, 'page_label': '55'}, page_content='M a d d ə  1 2 3 . Zəruri müdafiə həddini və ya cinayət törətmiş şəxsin tutulması üçün zəruri \nhəddi aşmaqla adam öldürmə \n

In [ ]:
question = "15 yaşında bir şəxs ağır cinayət törədərsə, Azərbaycan Respublikasının Cinayət Məcəlləsinə əsasən ona təyin edilə biləcək azadlıqdan məhrumetmə cəzasının maksimum müddəti nə qədərdir?"

response = graph.invoke({"question": question})
wrapped_answer = textwrap.fill(response["answer"], width=100)
print(wrapped_answer)
print("Sources:", response["context"])

Azərbaycan Respublikasının Cinayət Məcəlləsinə əsasən, 15 yaşında bir şəxs ağır cinayət törədərsə,
ona təyin edilə biləcək azadlıqdan məhrumetmə cəzasının maksimum müddəti 10 ildir. Yetkinlik yaşına
çatmayan məhkumlar cəzalarını tərbiyə müəssisələrində çəkirlər. Bu müəssisələr ümumi və möhkəm
rejimli olmaqla iki yerə bölünür.
Sources: [Document(id='b4ca9ac7-4f50-4045-a7f2-99a8d0c86bac', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-10-15T11:02:03+04:00', 'author': 'Novruz Ferhadov', 'moddate': '2019-10-15T11:02:03+04:00', 'source': '/content/drive/MyDrive/cinayet-mecellesi-pdf.pdf', 'total_pages': 300, 'page': 36, 'page_label': '37'}, page_content='84.1. Cinayət törədərkən on dörd yaşı tamam olmuş, lakin on səkkiz yaşına çatmayan şəxslər \nyetkinlik yaşına çatmayanlar hesab olunur. \n84.2. Yetkinlik yaşına çatmayanlar cinayət törətdikdə, onlara cəza təyin edilə bilər və ya tərbiyəvi \nxarakterli məcburi tədbirlər tətbiq edilə bil

# Multi Document Retrieval

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_community.document_loaders import PyPDFLoader

# Load and chunk contents of both documents
file_path_1 = "/content/drive/MyDrive/emek-mecellesi-1-20.pdf"
file_path_2 = "/content/drive/MyDrive/cinayet-mecellesi-pdf.pdf"

# Load first document
loader_1 = PyPDFLoader(file_path_1)
docs_1 = loader_1.load()

# Add metadata to identify source
for doc in docs_1:
    doc.metadata["source_document"] = "emek-mecellesi-1-20.pdf"
    doc.metadata["document_type"] = "labour_code"

# Load second document
loader_2 = PyPDFLoader(file_path_2)
docs_2 = loader_2.load()

# Add metadata to identify source
for doc in docs_2:
    doc.metadata["source_document"] = "cinayet-mecellesi-pdf.pdf"
    doc.metadata["document_type"] = "criminal_code"

# Combine all documents
all_docs = docs_1 + docs_2

# Split all documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1536, chunk_overlap=305)
all_splits = text_splitter.split_documents(all_docs)

# Index chunks - all documents go into the same vector store
_ = vector_store.add_documents(documents=all_splits)

prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    # Retrieve more documents since we have two sources
    retrieved_docs = vector_store.similarity_search(state["question"], k=6)
    return {"context": retrieved_docs}

def generate(state: State):
    # Include source information in the context
    docs_content = "\n\n".join(
        f"[Source: {doc.metadata.get('source_document', 'Unknown')}]\n{doc.page_content}"
        for doc in state["context"]
    )
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
response_1 = graph.invoke({"question": "Azərbaycan Respublikasında dövlət maliyyəli qurumlarda işləmək üçün qanuni yaş həddi nədir və elmi sahələr üçün bu qaydanın hansı istisnaları var?"})
print("Labour question:", response_1["answer"])

Labour question: Azərbaycan Respublikasında dövlət büdcəsindən maliyyələşən müəssisələrdə işləmənin yaş həddi 65 yaşdır. Lakin, bu qayda elmi müəssisə və təşkilatlar, ali təhsil müəssisələrinə şamil edilmir. Elm, mədəniyyət, səhiyyə və təhsil sahələrinin inkişafında xüsusi xidmətləri olmuş işçilərin işləmə müddəti müvafiq dövlət orqanı tərəfindən uzadıla bilər.


In [ ]:
question = "Cinayətlərin təsnifatı hansı əsaslara görə aparılır və cinayət hansı hallarda ideal məcmunu yaradır?"
response = graph.invoke({"question": question})
print(response["answer"])

Cinayətlərin təsnifatı xarakterindən və ictimai təhlükəlilik dərəcəsindən asılı olaraq aparılır. Bir hərəkətlə (hərəkətsizliklə) Cinayət Məcəlləsinin iki və daha çox maddəsi ilə nəzərdə tutulmuş iki və daha çox cinayətin törədilməsi cinayətlərin ideal məcmusunu yaradır.


In [ ]:
question = "Əmək Məcəlləsində məcburi əməyin qadağan edilməsi necə tənzimlənir və Cinayət Məcəlləsində məcburi əməyə görə hansı cəza nəzərdə tutulur?"
response = graph.invoke({"question": question})
print(response["answer"])

print("---------------")

print(response["context"])

Əmək Məcəlləsinin 17-ci maddəsinə əsasən, işçini zor işlətməklə və ya əmək müqaviləsinə xitam veriləcəyi hədə-qorxusu ilə əmək funksiyasına daxil olmayan işi yerinə yetirməyə məcbur etmək qadağandır. Cinayət Məcəlləsinin 144-2.1-ci maddəsi ilə hədə-qorxu, zor tətbiq etməklə və ya zor tətbiq etmək hədəsi ilə, habelə qanunvericiliklə müəyyən edilmiş xüsusi hallardan başqa şəxsin azadlığını məhdudlaşdırmaqla müəyyən işin yerinə yetirilməsinə məcbur etmə dörd ildən səkkiz ilədək müddətə azadlıqdan məhrum etmə ilə cəzalandırılır. Eyni əməllər iki və ya daha çox şəxs barəsində törədildikdə və ya təkrar törədildikdə daha ağır cəzalar nəzərdə tutulur.
---------------
[Document(id='72528fc3-71a0-42cf-afe3-f7fd242fff88', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-10-15T11:02:03+04:00', 'author': 'Novruz Ferhadov', 'moddate': '2019-10-15T11:02:03+04:00', 'source': '/content/drive/MyDrive/cinayet-mecellesi-pdf.pdf', 'total_pages': 300, 'p

# Testing


In [ ]:
import langextract as lx
import textwrap
import re # Import the regular expression module

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
from langchain_community.document_loaders import PyPDFLoader

# file_path = "/content/drive/MyDrive/Labour-Code-of-the-Republic-of-Azerbaijan.pdf"
file_path = "/content/drive/MyDrive/emek-mecellesi-1-2.pdf"

loader = PyPDFLoader(file_path)
docs = loader.load()

for doc in docs:
    # Remove all newlines and normalize whitespace
    doc.page_content = re.sub(r'\n+', ' ', doc.page_content)
    # Clean up multiple spaces
    doc.page_content = re.sub(r'\s+', ' ', doc.page_content).strip()



In [ ]:
import pymupdf4llm

import re

def clean_extracted_text(text):
  """
  Corrects common text extraction artifacts from a PDF to Markdown conversion.
  """
  # First, remove the double asterisks used for bolding.
  text = text.replace('**', '')

  # Next, remove underscore-space combinations that break up words.
  text = text.replace('_ ', '')
  text = text.replace(' _', '')

  # Finally, remove any remaining underscores.
  text = text.replace('_', '')

  text = re.sub(r'\[\d+\]', '', text)

  text = re.sub(r'\[[\d+\]]', '', text)

  text = text.strip()
  return text

md_text = pymupdf4llm.to_markdown("/content/drive/MyDrive/emek-mecellesi-1-2.pdf")
clean_extracted_text = clean_extracted_text(md_text)

In [ ]:
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1536, chunk_overlap=305)
# Create a Document object from the cleaned text
doc = Document(page_content=clean_extracted_text)
all_splits = text_splitter.split_documents([doc])

# Index chunks
_ = vector_store.add_documents(documents=all_splits)


prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
question = "Azərbaycan Respublikasının Əmək Məcəlləsi hansı iş yerlərinə və hansı şəxslərə şamil edilmir?"

response = graph.invoke({"question": question})
print(response["context"])
print(response["answer"])

[Document(id='6fa92791-71e0-4eff-b475-397c82bdc594', metadata={}, page_content='qurğularında və digər iş yerlərində tətbiq edilir ~~.~~\n\n2. Bu Məcəllə işəgötürənin xammalından (materialından), istehsal vasitələrindən istifadə etməklə əmək\nfunksiyasını öz evində yerinə yetirən işçilərə də şamil edilir.\n\n3. Azərbaycan Respublikasının qanunlarında müəyyən edilmişdövlət büdcəsindən maliyyələşən müəssisələrdəçalışmanın yaşhəddi elmi\n\n\n\nmüəssisəvətəşkilatlar, ali təhsil müəssisələrinəşamil edilmir.\n\n\n\n\n\n\n\n\n4.Ələt azad iqtisadi zonasındaəmək münasibətləri “Ələt azad iqtisadi zonası haqqında” Azərbaycan Respublikası Qanununun tələblərinə\n\n\n\nuyğun olaraq tənzimlənir.\n\n\n\n\n\n\n\n\nMaddə 5. Bu Məcəllənin tətbiq edildiyi digər iş yerləri və qulluqçular\n\n1. Azərbaycan Respublikasının xarici dövlətlərlə, beynəlxalq təşkilatlarla bağladığı müqavilələrində başqa\nhal nəzərdə tutulmayıbsa, bu Məcəllə müvafiq xarici dövlətlərin, onların fiziki və hüquqi şəxslərinin,\nbeynəlxa

In [ ]:
from langchain.text_splitter import MarkdownTextSplitter, MarkdownHeaderTextSplitter
from docling.document_converter import DocumentConverter
import re

source = "/content/drive/MyDrive/emek-mecellesi-1-20.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
markdown_content = result.document.export_to_markdown()

markdown_content = re.sub(r'\s*([əşŞğĞ])\s*', r'\1', markdown_content)
markdown_content = re.sub(r'([ə])([ABCÇDEƏFGHÜPRSŞTLNZTİXJKQMÖOUVY])', r'\1 \2', markdown_content)
markdown_content = re.sub(r'\[\d+\]', '', markdown_content)

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_content)

for doc in md_header_splits:
    print(doc.metadata)
    print(doc.page_content)
    print("-" * 20) # Separator for clarity

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'Header 2': 'AZƏRBAYCAN RESPUBLİKASININ ƏMƏK MƏCƏLLƏSİ'}
Azərbaycan Respublikası Konstitusiyasının 35-ci maddəsinəəsasənəmək fərdi vəictimai rifahınəsasıdır. Hər kəsinəməyəolan qabiliyyətiəsasında sərbəst surətdəözünəfəaliyyət növü, peşə, məşğuliyyət vəişyeri seçmək hüququ vardır.  
Azərbaycan Respublikasının Əmək Məcəlləsində:  
müvafiq  hüquq  normaları  iləəmək  münasibətlərindəişçilərin  vəişəgötürənlərinəmək,  sosial,  iqtisadi hüquqları vəbu hüquqlarla bağlı müvafiq təminatların minimum səviyyəsi;  
Azərbaycan Respublikası Konstitusiyasının ikinci bölməsindənəzərdətutulanəmək, istirahət, təhlükəsiz vəsağlamşəraitdəişləmək  hüququnun,  habelədigərəsas  insan  hüquqlarının  vəazadlıqlarının  təmin  edilməsi prinsipləri vəqaydaları;  
Azərbaycan  Respublikasının  bağladığı  vəya  tərəfdar  çıxdığı  beynəlxalq  müqavilələrə,  Beynəlxalq  Əmək Təşkilatının konvensiyalarına vədigər beynəlxalq hüquq normalarına uyğun olaraqəmək münasibətlərinin yaranması, dəyişdirilməsi, onlara xitam v

In [ ]:
len(md_header_splits)

82

In [ ]:
_ = vector_store.add_documents(documents=md_header_splits)


prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
question = "Həmkarlar İttifaqından könüllü şəkildə çıxa bilərəmmi?"

response = graph.invoke({"question": question})
print(response["context"])
print(response["answer"])

[Document(id='4c43f286-a1a4-4b7c-810d-2eccf582404a', metadata={'Header 2': ''}, page_content='- 1 .  Bu  Məcəllənin 70-ci maddəsinin b) vəç)  bəndlərindəgöstərilənəsaslarla işəgötürən tərəfindən həmkarlar ittifaqının üzvü olan işçininəmək müqaviləsi müəssisədəfəaliyyət göstərən həmkarlar ittifaqı təşkilatının qabaqcadan razılığı alınmaqla ləğv edilir.\n- 2 .  Həmkarlar  ittifaqının  üzvü  olan  işçininəmək  müqaviləsini  bu  maddənin  1-ci  hissəsindənəzərdətutulan  hallardan  hər  hansı  biri  iləəlaqədar  ləğv  etmək  istəyən  işəgötürən  həmin  müəssisənin  həmkarlar  ittifaqı  təşkilatınaəsaslandırılmışyazılı  təqdimatla  müraciət  edir. Təqdimata müvafiqəsaslandırma sənədləriəlavəedilir. Həmkarlar ittifaqı təşkilatı bu təqdimatın daxil olduğu gündənən geci on gün müddətində  \nözününəsaslandırılmışyazılı qərarını işəgötürənətəqdim etməlidir.  \n3. Bu maddənin birinci hissəsindənəzərdətutulmuşhallar istisna olunmaqla qalan hallardaəmək müqaviləsi işəgötürənin  təşəbbüsü  ilələğv  e